In [116]:
import pandas as pd
import numpy as np
import os
from datetime import date,timedelta

In [117]:
folder = r'C:\Users\jcallomamanib\Documents\GitProjects\maintenanceApp\02. Data2HR\output data'

asset_data = r'dataAD.xlsx'
asset_system = r'dataAS.xlsx'
system_component = r'dataSC.xlsx'
component_dutie = r'dataCD.xlsx'
dutie_dataM = r'dataDD_M.xlsx'
dutie_dataP = r'dataDD_P.xlsx'

ruteA_D = os.path.join(folder,asset_data)
ruteA_S = os.path.join(folder,asset_system)
ruteS_C = os.path.join(folder,system_component)
ruteC_D = os.path.join(folder,component_dutie)
ruteD_DM = os.path.join(folder,dutie_dataM)
ruteD_DP = os.path.join(folder,dutie_dataP)


In [130]:
%%time
dataA_D = pd.read_excel(ruteA_D, dtype = {'Cod sap':str})
dataA_S = pd.read_excel(ruteA_S, dtype = {'System':str})
dataS_C = pd.read_excel(ruteS_C)
dataC_D = pd.read_excel(ruteC_D)
dataD_DM = pd.read_excel(ruteD_DM, dtype = {'Specialist':str,'Amount Craft':int,'Time':float})
dataD_DP = pd.read_excel(ruteD_DP, dtype = {'Specialist':str,'Amount Craft':int,'Time':float})
dataD_DM.loc[:,'Last date task done']=pd.to_datetime(dataD_DM.loc[:,'Last date task done'],format='%d/%m/%Y')
dataD_DP.loc[:,'Last date task done']=pd.to_datetime(dataD_DP.loc[:,'Last date task done'],format='%d/%m/%Y')

Wall time: 2.13 s


In [131]:
## DATA OF DUTIES AND TASK INFORMATION
dataD_DM.loc[:,'Man hour'] = dataD_DM.loc[:,'Time']*dataD_DM.loc[:,'Amount Craft']
dataD_DP.loc[:,'Man hour'] = dataD_DP.loc[:,'Time']*dataD_DP.loc[:,'Amount Craft']

rowfilterM = ~dataD_DM.loc[:,'Frequency'].isnull()
rowfilterP = ~dataD_DP.loc[:,'Frequency'].isnull()

dataD_DM.loc[rowfilterM,'Strategy'] = dataD_DM.loc[rowfilterM,'Frequency'].astype(int).astype(str) + dataD_DM.loc[rowfilterM,'Unidad Periodo']

dataD_DP.loc[rowfilterP,'Strategy'] = dataD_DP.loc[rowfilterP,'Frequency'].astype(int).astype(str) + dataD_DP.loc[rowfilterP,'Unidad Periodo']

# All the conditional activities are remove (1T: first task: all activities with frequency 2T second task: all conditional activities)
dataD_DM_1T = dataD_DM.loc[dataD_DM.loc[:,'Maintenance type'] != 'COND',:].reset_index(drop = True)
dataD_DP_1T = dataD_DP.loc[dataD_DP.loc[:,'Maintenance type'] != 'COND',:].reset_index(drop = True)

In [133]:
%%time
# Merging all datas csv 
# Asset&Data(Johan&Jorge) -> Asset&System -> System&Component -> Component&Duties -> 
# Duties&Data(Mechanical/Predective/Mechanical&Predective)
dataA_D_S = pd.merge(dataA_D,dataA_S,how = 'left',on = 'Asset tag')
dataA_S_C = pd.merge(dataA_D_S, dataS_C, how='left', on = 'key_system')
dataA_S_C_D = pd.merge(dataA_S_C, dataC_D, how='left', on='key_component')
# Merging all with conditional activities
dataASCD_M = pd.merge(dataA_S_C_D, dataD_DM, how = 'inner', on = 'key_dutie')
dataASCD_P = pd.merge(dataA_S_C_D, dataD_DP, how = 'inner', on = 'key_dutie')
# Merging all without conditional activities, just first task
dataASCD_1TM = pd.merge(dataA_S_C_D, dataD_DM_1T, how = 'inner', on = 'key_dutie')
dataASCD_1TP = pd.merge(dataA_S_C_D, dataD_DP_1T, how = 'inner', on = 'key_dutie')


Wall time: 51.6 ms


In [134]:
# Adding the pivot column to project
dataASCD_1TM.loc[:,'pivot'] = dataASCD_1TM.loc[:,'Asset tag'] + '-' + dataASCD_1TM.loc[:,'Strategy'] 
dataASCD_1TP.loc[:,'pivot'] = dataASCD_1TP.loc[:,'Asset tag'] + '-' + dataASCD_1TP.loc[:,'Strategy']

In [135]:
columnfilter = ['key_system', 'Component','Primary task', 'Maintenance type', 'Specialist', 'Constraint', 'Time', 'Amount Craft','Frequency', 'Unidad Periodo','Strategy', 'Date task done']

In [136]:
projectionM = dataASCD_1TM.loc[:,['Asset tag','Strategy','Frequency', 'Unidad Periodo','Last date task done','Hour day']].groupby(['Asset tag','Strategy','Frequency', 'Unidad Periodo','Last date task done','Hour day']).size().reset_index()
projectionM.drop(columns = {0},inplace =True)
projectionM.loc[:,'Date plan'] = projectionM.loc[:,'Last date task done']
[row,column] = projectionM.shape

In [125]:
%%time
planDatesM = pd.DataFrame(columns = projectionM.columns)
projectionflag = projectionM

for m in range(row):
    while projectionflag.loc[m,'Date plan'] < date(2022,1,1):
        hour2days = round(projectionflag.loc[m,'Frequency']/projectionflag.loc[m,'Hour day'],0)
        if projectionflag.loc[m,'Unidad Periodo'] == 'H':
            delta=timedelta(days = hour2days)
        elif  projectionflag.loc[m,'Unidad Periodo'] == 'S':
            delta = timedelta(weeks = projectionflag.loc[m,'Frequency'])
        elif  projectionflag.loc[m,'Unidad Periodo'] == 'M':
            delta = timedelta(weeks = 4*projectionflag.loc[m,'Frequency'])
        elif  projectionflag.loc[m,'Unidad Periodo'] == 'A':
            delta = timedelta(weeks = 52*projectionflag.loc[m,'Frequency'])
        
        projectionflag.loc[m,'Date plan'] = projectionflag.loc[m,'Date plan'] + delta
        planDatesM = planDatesM.append(projectionflag.loc[m,:],ignore_index=True)
planDatesM.loc[:,'pivot'] = planDatesM.loc[:,'Asset tag'] + '-' + planDatesM.loc[:,'Strategy']

Wall time: 28.2 s


In [126]:
%%time
dataASCD_1TMPlan = pd.merge(dataASCD_1TM,planDatesM.loc[:,['pivot','Date plan']],how='left',on='pivot')
dataASCD_1TMPlan.to_excel('dataASCD_1TMPlan.xlsx', encoding='utf-8-sig',index=False)

Wall time: 21.1 s


In [127]:
projectionP = dataASCD_1TP.loc[:,['Asset tag','Strategy','Frequency', 'Unidad Periodo','Last date task done','Hour day']].groupby(['Asset tag','Strategy','Frequency', 'Unidad Periodo','Last date task done','Hour day']).size().reset_index()
projectionP.drop(columns = {0},inplace =True)
projectionP.loc[:,'Date plan'] = projectionP.loc[:,'Last date task done']
[row,column] = projectionP.shape

In [128]:
%%time
planDatesP = pd.DataFrame(columns = projectionP.columns)
projectionflag = projectionP

for m in range(row):
    while projectionflag.loc[m,'Date plan'] < date(2022,1,1):
        hour2days = round(projectionflag.loc[m,'Frequency']/projectionflag.loc[m,'Hour day'],0)
        if projectionflag.loc[m,'Unidad Periodo'] == 'H':
            delta=timedelta(days = hour2days)
        elif  projectionflag.loc[m,'Unidad Periodo'] == 'S':
            delta = timedelta(weeks = projectionflag.loc[m,'Frequency'])
        elif  projectionflag.loc[m,'Unidad Periodo'] == 'M':
            delta = timedelta(weeks = 4*projectionflag.loc[m,'Frequency'])
        elif  projectionflag.loc[m,'Unidad Periodo'] == 'A':
            delta = timedelta(weeks = 52*projectionflag.loc[m,'Frequency'])
        
        projectionflag.loc[m,'Date plan'] = projectionflag.loc[m,'Date plan'] + delta
        planDatesP = planDatesP.append(projectionflag.loc[m,:],ignore_index=True)
planDatesP.loc[:,'pivot'] = planDatesP.loc[:,'Asset tag'] + '-' + planDatesP.loc[:,'Strategy']

Wall time: 6.32 s


In [129]:
%%time
dataASCD_1TPPlan = pd.merge(dataASCD_1TP,planDatesP.loc[:,['pivot','Date plan']],how='left',on='pivot')
dataASCD_1TPPlan.to_excel('dataASCD_1TPPlan.xlsx', encoding='utf-8-sig',index=False)

Wall time: 5.25 s
